In [1]:
"""
Test stacking with labels and some convolutions.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import utils
import tensorflow as tf
from config import cfg
import conv_bodies
import modeling
import matplotlib.pyplot as plt
    

model, weights_file, start_iter, checkpoints, output_dir = modeling.create_model()
model.set_dataset('train_data/', 'train_labels/')


init = (tf.global_variables_initializer(), 
        tf.local_variables_initializer(), 
        model.dataset[1].make_initializer(model.dataset[0]))

model.tensor, model.labels = utils.stack_batch_into_tensor(model.dataset[0], train=True) # Normally use model.train

print(type(model.tensor))
with tf.Session() as sess:
    sess.run(init)
    
    with tf.variable_scope('conv1', reuse=tf.AUTO_REUSE):
        print(model.tensor.shape)
        model.Conv(3, 3, 64, padding='SAME', strides=[1,1,1,1])
        print(model.tensor.shape)
        model.Relu()
    with tf.variable_scope('conv2', reuse=tf.AUTO_REUSE):
        model.Conv(3, 64, 64, padding='SAME', strides=[1,1,1,1])
        print(model.tensor.shape)
        model.Relu()
        # Strides[0] must be 1, else
        # UnimplementError Pooling is not yet supported on the batch dimension
        model.MaxPool(ksize=[1,1,2,1], strides=[1,2,2,2], padding='SAME')
        print(model.tensor.shape)

    
    

<class 'tensorflow.python.framework.ops.Tensor'>
(12, 224, 224, 3)
(12, 224, 224, 64)
(12, 224, 224, 64)
(12, 112, 112, 32)


In [2]:
"""
Test loading datasets and iterating on them.
"""

dataset, iterator = utils.load_dataset('train_data/', 'train_labels/')
tensor = iterator.get_next()
next_element = iterator.get_next()
init = iterator.make_initializer(dataset)

with tf.Session() as sess:

    # Initialize the iterator on the training data
    sess.run(init)
    tensor = sess.run(tensor)[0]
    print(type(tensor))
    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            tensor = tf.stack((tensor, elem[0]), axis=0)
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            print(type(tensor))
            print(tensor.shape)
            break

<class 'numpy.ndarray'>
End of training dataset.
<class 'tensorflow.python.framework.ops.Tensor'>
(2, 224, 224, 3)


In [1]:
"""
Test stacking without labels and a convolutions.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import utils
import tensorflow as tf
from config import cfg
import conv_bodies
import modeling
    

model, weights_file, start_iter, checkpoints, output_dir = modeling.create_model()
model.set_dataset('train_data/')


init = (tf.global_variables_initializer(), 
            tf.local_variables_initializer(), 
            model.dataset[1].make_initializer(model.dataset[0]))

model.tensor = utils.stack_batch_into_tensor(model.dataset[0], train=False) # Normally use model.train

with tf.Session() as sess:
    sess.run(init)
    with tf.variable_scope('conv', reuse=tf.AUTO_REUSE):
        model.Conv(3, 3, 64, padding='SAME', strides=[1,1,1,1])
    
    

<class 'list'>
<class 'numpy.ndarray'>
12
